In [1]:
import time
import tensorflow as tf
#tf.debugging.set_log_device_placement(True)

In [2]:
print('-' * 61)
print('-' * 20, 'Individual training', '-' * 20)
print('-' * 61)

gpus = [gpu.name for gpu in tf.config.experimental.list_logical_devices('GPU')]
for dev in ['/CPU:0'] + gpus:
    print('Training on device:', dev)
    print()
    t = time.perf_counter()
    with tf.device(dev):
        mnist = tf.keras.datasets.mnist
        
        (x_train, y_train), (x_test, y_test) = mnist.load_data()
        x_train, x_test = x_train / 255.0, x_test / 255.0
        
        model = tf.keras.models.Sequential([
          tf.keras.layers.Flatten(input_shape=(28, 28)),
          tf.keras.layers.Dense(128, activation='relu'),
          tf.keras.layers.Dropout(0.2),
          tf.keras.layers.Dense(10)
        ])

        predictions = model(x_train[:1]).numpy()
        print(predictions)

        print(tf.nn.softmax(predictions).numpy())

        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

        print(loss_fn(y_train[:1], predictions).numpy())

        model.compile(optimizer='adam',
                      loss=loss_fn,
                      metrics=['accuracy'])

        model.fit(x_train, y_train, epochs=5)

        print(model.evaluate(x_test,  y_test, verbose=2))

        probability_model = tf.keras.Sequential([
          model,
          tf.keras.layers.Softmax()
        ])

        print(probability_model(x_test[:5]))
    
    print()
    print(dev + " took " + str(time.perf_counter() - t) + " seconds")
    print('-' * 61)

-------------------------------------------------------------
-------------------- Individual training --------------------
-------------------------------------------------------------
Training on device: /CPU:0

[[-0.04846171 -0.13890412 -0.02299292 -0.07997595  0.2928807  -0.31734282
  -0.49769437 -0.03430814 -0.21658596  0.24343552]]
[[0.10084756 0.09212697 0.10344902 0.09771899 0.14187588 0.07707115
  0.06435261 0.10228507 0.08524129 0.1350314 ]]
2.5630262
Epoch 1/5
1875/1875 [==============================] - 2s 733us/step - loss: 0.4757 - accuracy: 0.8567
Epoch 2/5
1875/1875 [==============================] - 1s 720us/step - loss: 0.1511 - accuracy: 0.9560
Epoch 3/5
1875/1875 [==============================] - 1s 708us/step - loss: 0.1058 - accuracy: 0.9674
Epoch 4/5
1875/1875 [==============================] - 1s 747us/step - loss: 0.0876 - accuracy: 0.9733
Epoch 5/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0747 - accuracy: 0.9762
313/313 - 0s - loss: 

In [3]:
print('-' * 61)
print('-' * 20, 'Parallel training', '-' * 22)
print('-' * 61)

t = time.perf_counter()
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
    model = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(10)
    ])

predictions = model(x_train[:1]).numpy()
print(predictions)

print(tf.nn.softmax(predictions).numpy())

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

print(loss_fn(y_train[:1], predictions).numpy())

model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

print(model.evaluate(x_test,  y_test, verbose=2))

probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

print(probability_model(x_test[:5]))
print()
print("Parallel took " + str(time.perf_counter() - t) + " seconds")
print('-' * 61)

-------------------------------------------------------------
-------------------- Parallel training ----------------------
-------------------------------------------------------------
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
[[ 0.6456045  -0.01259251 -0.6148372  -0.00272581 -1.0280677   0.393279
  -0.204669    0.503689   -0.12731948 -0.7685883 ]]
[[0.18908754 0.09790652 0.05361175 0.09887731 0.0354647  0.14691946
  0.08079674 0.16407023 0.08729438 0.0459713 ]]
1.9178708
Epoch 1/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.4850 - accuracy: 0.8591
Epoch 2/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1444 - accuracy: 0.9571
Epoch 3/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1060 - accuracy: 0.9679
Epoch 4/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0854 - accuracy: 0.9737
Epoch 5/5
1875/1875 [==============================] - 2s 